### Author: Guilherme Resende

This notebook is applies the best performing model and chunking approach.

In [ ]:
import json
import os

credentials=None
with open("credentials.json", 'r') as f:
    credentials = json.load(f)

os.environ["OPENAI_API_KEY"] = credentials["OPENAI_API_KEY"]

In [ ]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader, UnstructuredMarkdownLoader
from llama_index.core.evaluation import (
    FaithfulnessEvaluator,
    RelevancyEvaluator
)

In [ ]:
embedding = OpenAIEmbeddings()
persist_directory = "db"
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

model = OpenAI()

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=model, 
    chain_type="stuff", 
    retriever=retriever, 
    return_source_documents=True
)

In [ ]:
def process_llm_response(llm_response):
    print(llm_response['result'])
    
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
query = "How to deploy SageMaker at AWS?"
llm_response = qa_chain(query)
process_llm_response(llm_response)